# Frequency-domain ODMR measurement



![frequency-shceduling](./figure/frequency-shceduling.jpg)


Import necessary modules and functions firstly

In [1]:
import matplotlib.pyplot as plt
from odmactor.scheduler import CWScheduler, PulseScheduler
import scipy.constants as C
import numpy as np
import time


## CW ODMR

**1. Set measurement parameters**
- Time bin $t$ per readout operation, number of readout operation $N$ per frequency measurement point
- MW power $p$, MW frequencies for scanning modes, i.e., `[freq_start, freq_end, freq_step]`, unit: "Hz"


In [2]:
t_ns = 1e5
N = int(1e5 / 4)


freq_start = 2.85 * C.giga
freq_end = 2.9 * C.giga
freq_step = 1 * C.mega
p = 10


**2. Construct a Continous-wave Scheduler and run**


In [ ]:
scheduler = CWScheduler()
scheduler.configure_mw_paras(p)
scheduler.configure_odmr_seq(t_ns, N)
scheduler.set_mw_freqs(freq_start, freq_end, freq_step)
scheduler.configure_tagger_counting()

scheduler.run_scanning(with_ref=True)
scheduler.close()


**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
res_on_off = scheduler.result  # [freqs, counts, counts_ref]
contrast = [sig/ref for sig, ref in zip(res_on_off[1], res_on_off[2])]

In [ ]:
plt.plot(res_on_off[0], res_on_off[1],'o-',label='MW on')
plt.plot(res_on_off[0], res_on_off[2], 'o-', label='MW off')
plt.legend()
plt.title('Counts')
plt.show()

In [ ]:
plt.plot(res_on_off[0], contrast, 'o-')
plt.title('Contrast')
plt.show()

**4. Save data result**

In [ ]:

np.savetxt('cw-counting--{}dBm.txt'.format(p), np.array(res_on_off))
np.savetxt('cw-contrast--{}dBm.txt'.format(p),np.vstack([res_on_off[0], contrast]).T)


## Pulse ODMR


**1. Set measurement parameters**
- number of readout operation $N$ per frequency measurement point
- MW power $p$, MW frequencies for scanning modes, i.e., `[freq_start, freq_end, freq_step]`, unit: "Hz"
- laser initilization time $t_{init}$, MW operation time $t_{mw}$, singal readout pulse time $t_{sig}$
- optional default parameters: reference signals readout time $t_{ref}$ ($t_{ref} = t_{sig}$ actually), some time interval parameters
- Time bin $t$ per readout operation is then calculated by the sequences parameters

In [ ]:
freq_start = 2.85 * C.giga
freq_end = 2.89 * C.giga
freq_step = 2 * C.mega



t_init = 5e3
t_mw = 1e3
inter_init_mw = 3e3

N = int(1e6/2)

p = 15

**2. Construct a Pulse ODMR Scheduler and run**

In [ ]:
scheduler = PulseScheduler()
scheduler.configure_mw_paras(power=p)

# single-signal readout
scheduler.configure_odmr_seq(t_init, t_mw, t_read_sig=400, inter_init_mw=inter_init_mw, N=N)
# or readout with reference signals
# scheduler.configure_odmr_seq(t_init, t_mw, t_read_sig=400, t_read_ref=400,inter_init_mw=inter_init_mw, N=N)

scheduler.set_mw_freqs(freq_start, freq_end, freq_step)
scheduler.configure_tagger_counting(reader='cbm')

scheduler.run_scanning()

scheduler.close()



In [ ]:
# observe sequences
fig = scheduler.sequences_figure

**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
counts_sig_ref = scheduler.result  # [freqs, counts, counts_ref]

In [ ]:
plt.plot(counts_sig_ref[0], counts_sig_ref[1],'o-',label='Signal')
plt.plot(counts_sig_ref[0], counts_sig_ref[2], 'o-', label='Reference')
plt.legend()
plt.title('Counts')
plt.show()

In [ ]:
contrast = [sig/ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]

In [ ]:
plt.plot(counts_sig_ref[0], contrast, 'o-')
plt.title('Pulse ODMR Contrast')
plt.show()